<a href="https://colab.research.google.com/github/aaalexlit/omdena_climate_change_challenge_notebooks/blob/main/Triplets_from_Climate_FEVER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install datasets

In [42]:
from datasets import load_dataset
import pandas as pd
import json

In [10]:
ds_orig = load_dataset("climate_fever", split='test')
df = ds_orig.to_pandas()

In [9]:
df = ds_orig.to_pandas()

,claim_id,claim,claim_label,evidences
0,0,Global warming is driving polar bears toward e...,0,[{'evidence_id': 'Extinction risk from global ...
1,5,The sun has gone into ‘lockdown’ which could c...,0,"[{'evidence_id': 'Famine:386', 'evidence_label..."
2,6,The polar bear population has been growing.,1,"[{'evidence_id': 'Polar bear:1332', 'evidence_..."
3,9,Ironic' study finds more CO2 has slightly cool...,1,"[{'evidence_id': 'Atmosphere of Mars:131', 'ev..."
4,10,Human additions of CO2 are in the margin of er...,1,[{'evidence_id': 'Carbon dioxide in Earth's at...
...,...,...,...,...
1530,3125,About 60% of the warming observed from 1970 to...,2,"[{'evidence_id': 'Climate variability:103', 'e..."
1531,3127,"""Skeptics hope that Postma’s alternative therm...",2,"[{'evidence_id': 'Meteorology:4', 'evidence_la..."
1532,3130,"""There are other possible causes for climate c...",0,[{'evidence_id': 'Attribution of recent climat...
1533,3131,We don't need a high heat flow - just a high t...,2,"[{'evidence_id': 'Earth:161', 'evidence_label'..."


In [12]:
exploded = df.explode('evidences').reset_index(drop=True)
df_final = pd.concat([pd.json_normalize(exploded['evidences']), exploded], axis=1).drop(columns='evidences')

In [37]:
# take only disputed claims - that means that the claim has at least one definite SUPPORTS and REFUTES label
# and take only refuting/supporting evidences ie take out NEI ones
disputed_claims = df_final.query("claim_label == 3 and evidence_label != 2")[['evidence_id', 'evidence_label', 'evidence', 'claim_id', 'claim']]
# make sure that the first column is the evidence that supports and the second one that refutes 
triplets_df = disputed_claims.merge(disputed_claims, on="claim_id").query("evidence_label_x < evidence_label_y")
# select only relevant columns
triplets_df = triplets_df[["claim_id", "claim_x", "evidence_x", "evidence_y"]]
# rename columns
triplets_df.rename(columns={"claim_x":"claim", "evidence_x":"supports", "evidence_y":"refutes"}, inplace=True)
triplets_df

,claim_id,claim,supports,refutes
4,55,"[T]he raw data, the actual thermometer data[.....",He said there had probably been no global warm...,The reconstruction found significant variabili...
6,55,"[T]he raw data, the actual thermometer data[.....",He said there had probably been no global warm...,"In fact, if one ignores the unusual El Nino ye..."
7,55,"[T]he raw data, the actual thermometer data[.....",He said there had probably been no global warm...,That's the data we've had for the past 150 yea...
17,60,"So that means that probably about half, maybe ...","In 2018, Michaels asserted on Fox News, ""proba...",Most of the climatic warming over the last 50 ...
18,60,"So that means that probably about half, maybe ...","In 2018, Michaels asserted on Fox News, ""proba...",Human-caused increases in greenhouse gases are...
...,...,...,...,...
1674,3058,"Correspondingly, the global surface temperatur...",Earlier extended minima have been discovered t...,These isotope changes occurred due to the rele...
1678,3058,"Correspondingly, the global surface temperatur...",All these effects can combine to produce a dra...,These isotope changes occurred due to the rele...
1683,3072,"""a paper...in Science magazine concludes that ...",For constant humidity they computed a climate ...,"In his first paper on the matter, he estimated..."
1686,3084,"""the temperature increase in the second half o...","For example, one of the most recent results, e...","But, more accurately, global warming is the ma..."


In [45]:
result = triplets_df.to_json(orient="records")
parsed = json.loads(result)
print(json.dumps(parsed, indent=4))

[
    {
        "claim_id": "55",
        "claim": "[T]he raw data, the actual thermometer data[...] shows that the US has been cooling for 80 to 90 years.",
        "supports": "He said there had probably been no global warming since the 1940s, and \"Satellite data show no appreciable warming of the global atmosphere since 1979.",
        "refutes": "The reconstruction found significant variability around a long-term cooling trend of \u22120.02\u00a0\u00b0C per century, as expected from orbital forcing, interrupted in the 20th century by rapid warming which stood out from the whole period, with the 1990s \"the warmest decade, and 1998 the warmest year, at moderately high levels of confidence\"."
    },
    {
        "claim_id": "55",
        "claim": "[T]he raw data, the actual thermometer data[...] shows that the US has been cooling for 80 to 90 years.",
        "supports": "He said there had probably been no global warming since the 1940s, and \"Satellite data show no appreciable wa